# Variable elimination in Python using pgmpy

Notebook by Mika van Emmerloot  
Package `pgmpy` by Ankur Ankan (currently a PhD student in causal inference at Radboud)  
https://pgmpy.org/index.html

In [1]:
# Run this cell on Google Colab to install the pgmpy package
!pip install pgmpy

INFO: pip is looking at multiple versions of pgmpy to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
      --------------------------------------- 0.0/1.9 MB ? eta -:--:--
      --------------------------------------- 0.0/1.9 MB ? eta -:--:--
     --------------- ------------------------ 0.7/1.9 MB 5.8 MB/s eta 0:00:01
     ------------------------------------ --- 1.8/1.9 MB 9.3 MB/s eta 0:00:01
     ---------------------------------------- 1.9/1.9 MB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     --------------------- ------------------ 1.0/1.9 MB 33.4 MB/s eta 0:00:01
     --------------------- ------------------ 1.0/1.9 MB 33.4 MB/s eta 0:00:01
     --------------------- ------------------ 1.0/1.9 MB 33.4 MB/s eta 0:00:01
     ------------------------------ --------- 1.5/1.9 MB 8.4 MB/s eta 0:00:01
     --------------------

In [2]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

ModuleNotFoundError: No module named 'torch'

## Preliminaries

The order in which the truth values are specified follows the order you would expect to see in a truth table:

| Tampering |  f   |
|-----------|------|
| False     | 0.98 |
| True      | 0.02 |

---

| Alarm | Tampering | Fire  |   f    |
|-------|-----------|-------|--------|
| False |   False   | False | 0.9999 |
| False |   False   | True  | 0.0100 |
| False |   True    | False | 0.1500 |
| False |   True    | True  | 0.5000 |
| True  |   False   | False | 0.0001 |
| True  |   False   | True  | 0.9900 |
| True  |   True    | False | 0.8500 |
| True  |   True    | True  | 0.5000 |

---

| Leaving | Alarm |   f   |
|---------|-------|-------|
| False   | False | 0.999 |
| False   | True  | 0.120 |
| True    | False | 0.001 |
| True    | True  | 0.880 |

CPT's tend to be given with the "True" rows first (sometimes omitting the "False" rows altogether since these can be inferred), so pay attention to this 😁.

In [3]:
cpd_tampering = TabularCPD(
                variable='Tampering', variable_card=2,
                values=[[.98], [0.02]])
print(cpd_tampering)

cpd_alarm = TabularCPD(
            variable='Alarm', variable_card=2,
            values=[[0.9999, 0.0100, 0.1500, 0.5],
                    [0.0001, 0.9900, 0.8500, 0.5]],
            evidence=['Tampering', 'Fire'],
            evidence_card=[2, 2])
print(cpd_alarm)

cpd_leaving = TabularCPD(
              variable='Leaving', variable_card=2,
              values=[[0.999, 0.12], [0.001, 0.88]],
              evidence=['Alarm'], evidence_card=[2])
print(cpd_leaving)

+--------------+------+
| Tampering(0) | 0.98 |
+--------------+------+
| Tampering(1) | 0.02 |
+--------------+------+
+-----------+--------------+--------------+--------------+--------------+
| Tampering | Tampering(0) | Tampering(0) | Tampering(1) | Tampering(1) |
+-----------+--------------+--------------+--------------+--------------+
| Fire      | Fire(0)      | Fire(1)      | Fire(0)      | Fire(1)      |
+-----------+--------------+--------------+--------------+--------------+
| Alarm(0)  | 0.9999       | 0.01         | 0.15         | 0.5          |
+-----------+--------------+--------------+--------------+--------------+
| Alarm(1)  | 0.0001       | 0.99         | 0.85         | 0.5          |
+-----------+--------------+--------------+--------------+--------------+
+------------+----------+----------+
| Alarm      | Alarm(0) | Alarm(1) |
+------------+----------+----------+
| Leaving(0) | 0.999    | 0.12     |
+------------+----------+----------+
| Leaving(1) | 0.001    | 0.8

In the `TabularCPD` constructor, the `evidence` parameter is a list containing the names of the direct parents.  
The `variable_card` and `evidence_card` parameters denote the cardinality (= number of different values) of the node and its parents, respectively.
In this notebook I only use binary variables, so the cardinality will always be 2.

In [4]:
example_model = BayesianNetwork([
                ('Burglary', 'Alarm'),
                ('Earthquake', 'Alarm')])

print(example_model.nodes)
print(example_model.edges)
print(example_model.get_parents('Alarm'))

['Burglary', 'Alarm', 'Earthquake']
[('Burglary', 'Alarm'), ('Earthquake', 'Alarm')]
['Burglary', 'Earthquake']


As can be seen, the `BayesianNetwork` is constructed using a list of edges, where `(A, B)` denotes an edge $A \to B$, making $A$ a parent of $B$.

## Example 1: Earthquake

Here we have the bayesian network with the following edges:
- $Burglary \rightarrow Alarm$
- $Earthquake \rightarrow Alarm$
- $Alarm \rightarrow JohnCalls$
- $Alarm \rightarrow MaryCalls$

This network can be found at https://www.bnlearn.com/bnrepository/discrete-small.html#earthquake and is also given as a .bif file for the assignment.

In [5]:
earthquake_model =  BayesianNetwork([
                    ('Burglary', 'Alarm'),
                    ('Earthquake', 'Alarm'),
                    ('Alarm', 'JohnCalls'),
                    ('Alarm', 'MaryCalls')])

# Defining the parameters using CPT
cpd_burglary =  TabularCPD(
                variable='Burglary', variable_card=2,
                values=[[.999], [0.001]])

cpd_earthquake =  TabularCPD(
                  variable='Earthquake', variable_card=2,
                  values=[[0.98], [0.02]])

cpd_alarm = TabularCPD(
            variable='Alarm', variable_card=2,
            values=[[0.999, 0.71, 0.06, 0.05],
                    [0.001, 0.29, 0.94, 0.95]],
            evidence=['Burglary', 'Earthquake'],
            evidence_card=[2, 2])

cpd_johncalls = TabularCPD(
                variable='JohnCalls', variable_card=2,
                values=[[0.95, 0.1], [0.05, 0.9]],
                evidence=['Alarm'], evidence_card=[2])

cpd_marycalls = TabularCPD(
                variable='MaryCalls', variable_card=2,
                values=[[0.99, 0.3], [0.01, 0.7]],
                evidence=['Alarm'], evidence_card=[2])

# Associating the parameters with the model structure
earthquake_model.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_johncalls, cpd_marycalls)

print(earthquake_model)

BayesianNetwork with 5 nodes and 4 edges


In [6]:
inference = VariableElimination(earthquake_model)

# Calculate the CPT for Alarm, when observing Burglary to be true
result =  inference.query(
          variables=['Alarm'],
          evidence={'Burglary': True},
          elimination_order=['MaryCalls', 'JohnCalls', 'Earthquake'],
          joint=False)

for factor in result.values():
    print(factor)

  0%|          | 0/1 [00:00<?, ?it/s]

+----------+--------------+
| Alarm    |   phi(Alarm) |
+==========+==============+
| Alarm(0) |       0.0598 |
+----------+--------------+
| Alarm(1) |       0.9402 |
+----------+--------------+


## Example 2: Alarm

Here we have the bayesian network with the following edges:
- $Tampering \rightarrow Alarm$
- $Fire \rightarrow Alarm$
- $Fire \rightarrow Smoke$
- $Alarm \rightarrow Leaving$
- $Leaving \rightarrow Report$

This network can be found as the first example on http://artint.info/2e/html/ArtInt2e.Ch8.S3.SS2.html#Ch8.Thmciexamplered15 and should be very familiar as it has been used in the slides as well.

In [7]:
alarm_model = BayesianNetwork([
              ('Tampering', 'Alarm'),
              ('Fire', 'Alarm'),
              ('Fire', 'Smoke'),
              ('Alarm', 'Leaving'),
              ('Leaving', 'Report')])

# Defining the parameters using CPT
cpd_tampering = TabularCPD(
                variable='Tampering', variable_card=2,
                values=[[.98], [0.02]])

cpd_fire =  TabularCPD(
            variable='Fire', variable_card=2,
            values=[[0.99], [0.01]])

cpd_alarm = TabularCPD(
            variable='Alarm', variable_card=2,
            values=[[0.9999, 0.0100, 0.1500, 0.5],
                    [0.0001, 0.9900, 0.8500, 0.5]],
            evidence=['Tampering', 'Fire'],
            evidence_card=[2, 2])

cpd_smoke = TabularCPD(
            variable='Smoke', variable_card=2,
            values=[[0.99, 0.1], [0.01, 0.9]],
            evidence=['Fire'], evidence_card=[2])

cpd_leaving = TabularCPD(
              variable='Leaving', variable_card=2,
              values=[[0.999, 0.12], [0.001, 0.88]],
              evidence=['Alarm'], evidence_card=[2])

cpd_report =  TabularCPD(
              variable='Report', variable_card=2,
              values=[[0.99, 0.25], [0.01, 0.75]],
              evidence=['Leaving'], evidence_card=[2])

# Associating the parameters with the model structure
alarm_model.add_cpds(cpd_tampering, cpd_fire, cpd_alarm, cpd_smoke, cpd_leaving, cpd_report)

print(alarm_model)

BayesianNetwork with 6 nodes and 5 edges


In [8]:
inference = VariableElimination(alarm_model)

# Calculate the CPT for Tampering, when observing Smoke to be true and Report to be true
result =  inference.query(
          variables=['Tampering'],
          evidence={'Smoke': True, 'Report':True},
          elimination_order=['Leaving', 'Fire', 'Alarm'],
          joint=False)

for factor in result.values():
    print(factor)

  0%|          | 0/3 [00:00<?, ?it/s]

+--------------+------------------+
| Tampering    |   phi(Tampering) |
+==============+==================+
| Tampering(0) |           0.9716 |
+--------------+------------------+
| Tampering(1) |           0.0284 |
+--------------+------------------+


## Example 3: Alarm

Here we have the bayesian network with the following edges:
- $A \rightarrow C$
- $B \rightarrow C$
- $B \rightarrow D$
- $C \rightarrow E$
- $C \rightarrow F$

This is the network from the knowledge clip on Brightspace.

In [9]:
clip_model =  BayesianNetwork([
              ('A', 'C'),
              ('B', 'C'),
              ('B', 'D'),
              ('C', 'E'),
              ('C', 'F')])

# Defining the parameters using CPT
cpd_a = TabularCPD(
        variable='A', variable_card=2,
        values=[[0.1], [0.9]])

cpd_b = TabularCPD(
        variable='B', variable_card=2,
        values=[[0.8], [0.2]])

cpd_c = TabularCPD(
        variable='C', variable_card=2,
        values=[[0.6, 0.3, 0.2, 0.9],
                [0.4, 0.7, 0.8, 0.1]],
        evidence=['A', 'B'],
        evidence_card=[2, 2])

cpd_d = TabularCPD(
        variable='D', variable_card=2,
        values=[[0.2, 0.9], [0.8, 0.1]],
        evidence=['B'], evidence_card=[2])

cpd_e = TabularCPD(
        variable='E', variable_card=2,
        values=[[0.8, 0.3], [0.2, 0.7]],
        evidence=['C'], evidence_card=[2])

cpd_f = TabularCPD(
        variable='F', variable_card=2,
        values=[[0.1, 0.8], [0.9, 0.2]],
        evidence=['C'], evidence_card=[2])

# Associating the parameters with the model structure
clip_model.add_cpds(cpd_a, cpd_b, cpd_c, cpd_d, cpd_e, cpd_f)

print(clip_model)

BayesianNetwork with 6 nodes and 5 edges


In [10]:
inference = VariableElimination(clip_model)

# Calculate the CPT for E, when not observing anything
result =  inference.query(
          variables=['E'],
          evidence=None,
          elimination_order=['D', 'F', 'A', 'B', 'C'],
          joint=False)

for factor in result.values():
    print(factor)

  0%|          | 0/3 [00:00<?, ?it/s]

+------+----------+
| E    |   phi(E) |
+======+==========+
| E(0) |   0.4800 |
+------+----------+
| E(1) |   0.5200 |
+------+----------+
